In [7]:
import pickle
import numpy as np
import pandas as pd

In [8]:
with open('../batch1.pkl', 'rb') as f:
    batch1 = pickle.load(f)

In [9]:
def extract_features(bat_dict, early_cycles=[25, 50, 100]):
    """
    Her pil için farklı döngü sayılarından özellikleri çıkar
    """
    features_list = []
    
    for cell_id, cell_data in bat_dict.items():
        cycle_life = cell_data['cycle_life'][0]  # Hedef değişken
        
        for n_cycles in early_cycles:
            # İlk n döngüden özellikleri çıkar
            summary = cell_data['summary']
            
            features = {
                'cell_id': cell_id,
                'cycle_life': cycle_life,
                'n_cycles': n_cycles,
                'Qd_mean': np.mean(summary['QD'][:n_cycles]),
                'Qd_std': np.std(summary['QD'][:n_cycles]),
                'IR_mean': np.mean(summary['IR'][:n_cycles]),
                'IR_std': np.std(summary['IR'][:n_cycles]),
                'Tavg_mean': np.mean(summary['Tavg'][:n_cycles]),
                'dQd_slope': np.polyfit(range(n_cycles), summary['QD'][:n_cycles], 1)[0],  # Kapasite düşüş hızı
            }

            features_list.append(features)
    
    return pd.DataFrame(features_list)


In [10]:
def extract_capacity_fade_features(cell_data, n_cycles):
    """
    İlk n döngüdeki discharge capacity (Qd) değerlerinden
    kapasite düşüş eğimi (slope) ve intercept çıkarır.
    """
    cycles = cell_data["cycles"]

    Qd_values = []
    cycle_nums = []

    max_cycle = min(n_cycles, len(cycles))

    for i in range(max_cycle):
        key = str(i)
        if key not in cycles:
            continue
        
        Qd = np.array(cycles[key]["Qd"])
        if Qd.size == 0:
            continue
        
        Qd_mean = np.mean(Qd)
        Qd_values.append(Qd_mean)
        cycle_nums.append(i)

    if len(Qd_values) < 2:
        return {
            "Qd_slope": np.nan,
            "Qd_intercept": np.nan,
            "Qd_first": np.nan,
            "Qd_last": np.nan,
            "Qd_delta": np.nan,
        }

    cycle_nums = np.array(cycle_nums, dtype=float)
    Qd_values = np.array(Qd_values, dtype=float)

    slope, intercept = np.polyfit(cycle_nums, Qd_values, 1)

    return {
        "Qd_slope": float(slope),
        "Qd_intercept": float(intercept),
        "Qd_first": float(Qd_values[0]),
        "Qd_last": float(Qd_values[-1]),
        "Qd_delta": float(Qd_values[-1] - Qd_values[0]),
    }


In [11]:
df_batch1 = extract_features(batch1)

In [12]:
df_all = pd.concat([df_batch1], ignore_index=True)
df_all.to_csv('features_early_cycles.csv', index=False)